In [1]:
from os.path import join as pjoin, isdir, basename, isfile
from os import listdir, mkdir
import numpy as np
from glob import glob
from tqdm import tqdm 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
data = pd.read_excel('PPD_RTW_summary_table_Wei_06may2022_V2.xlsx')
data.head(10)

,session,Cell location,Cell ID,Stimulus_related_firing_P_value,Movement_related_firing_P_value,Reward_related_firing_P_value,Background_firing_correlation_Coefficient_with_PPD,Background_firing_correlation_P_ value_with PPD,Response_magnitude_correlation_Coefficient_with_PPD,Response_magnitude_correlation_P_ value_with PPD
0,AKED01_21.07.21,PFC,1,2.591914e-01,6.587486e-01,0.843829,-0.142275,0.007769,-0.186025,0.000477
1,AKED01_21.07.21,PFC,2,3.293294e-15,5.057776e-12,0.262134,-0.071628,0.181862,0.036298,0.499111
2,AKED01_21.07.21,PFC,3,4.966083e-05,1.240931e-06,0.649843,-0.189679,0.000366,-0.251533,0.000002
3,AKED01_21.07.21,PFC,4,4.782654e-03,3.283580e-02,0.546786,-0.086678,0.105982,-0.115271,0.031327
4,AKED01_21.07.21,PFC,5,2.599262e-03,1.008144e-02,0.144231,-0.208114,0.000090,-0.243660,0.000004
5,AKED01_21.07.21,PFC,6,5.588068e-01,2.491873e-01,0.062508,-0.056258,0.294621,-0.134867,0.011668
6,AKED01_21.07.21,PFC,7,2.225593e-11,1.371979e-14,0.326875,0.023332,0.664023,0.170237,0.001412
7,AKED01_21.07.21,PFC,8,2.947687e-02,1.189320e-03,0.168684,-0.020356,0.704719,0.046154,0.390009
8,AKED01_21.07.21,PFC,9,9.605673e-01,1.994682e-01,0.303200,-0.157429,0.003189,-0.198426,0.000191
9,AKED01_21.07.21,PFC,10,4.268626e-01,3.475549e-01,0.053469,-0.065312,0.223585,-0.071659,0.181678


In [12]:
mice = []
dates = []
str_names = []
pfc_names = []

Stimulus_related_firing_P_value_str = []
Movement_related_firing_P_value_str = []
Reward_related_firing_P_value_str = []
BG_firing_CC_with_PPD_str = []
BG_firing_CC_with_PPD_P_value_str= []
Response_magnitude_CC_with_PPD_str = []
Response_magnitude_CC_with_PPD_P_value_str = []

Stimulus_related_firing_P_value_pfc = []
Movement_related_firing_P_value_pfc = []
Reward_related_firing_P_value_pfc = []
BG_firing_CC_with_PPD_pfc = []
BG_firing_CC_with_PPD_P_value_pfc = []
Response_magnitude_CC_with_PPD_pfc = []
Response_magnitude_CC_with_PPD_P_value_pfc = []

peak = []
peak_width = []
spike_count_in_peak = []

for qualified_image in listdir(pjoin('data', 'PMSE', 'qualified')):
    pair_name = qualified_image.split('.')[0]
    session_name, s, str_code, p, pfc_code = pair_name.split('_')
    spike_count_data = pd.read_csv(pjoin('data', 'PMSE', f'{session_name}.csv'))
    str_name = '_'.join([s, str_code])
    pfc_name = '_'.join([p, pfc_code])
    mouse = session_name[:6]
    date = session_name[6:]
    year = date[2:4]
    month = date[4:6]
    day = date[6:]
    session_name_wei = '_'.join([mouse, '.'.join([year, month, day])])

    str_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='STR') & (data['Cell ID']==int(str_code)+1)]
    pfc_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='PFC') & (data['Cell ID']==int(pfc_code)+1)]

    if (str_row['Response_magnitude_correlation_P_ value_with PPD'].item() < 0.01 and 
       pfc_row['Response_magnitude_correlation_P_ value_with PPD'].item() < 0.01):
        mice.append(mouse)
        dates.append(date)
        str_names.append(f'str_{int(str_code) + 1}')
        pfc_names.append(f'pfc_{int(pfc_code) + 1}')

        Stimulus_related_firing_P_value_str.append(str_row['Stimulus_related_firing_P_value'].item())
        Movement_related_firing_P_value_str.append(str_row['Movement_related_firing_P_value'].item())
        Reward_related_firing_P_value_str.append(str_row['Reward_related_firing_P_value'].item())
        BG_firing_CC_with_PPD_str.append(str_row['Background_firing_correlation_Coefficient_with_PPD'].item())
        BG_firing_CC_with_PPD_P_value_str.append(str_row['Background_firing_correlation_P_ value_with PPD'].item())
        Response_magnitude_CC_with_PPD_str.append(str_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
        Response_magnitude_CC_with_PPD_P_value_str.append(str_row['Response_magnitude_correlation_P_ value_with PPD'].item())

        Stimulus_related_firing_P_value_pfc.append(pfc_row['Stimulus_related_firing_P_value'].item())
        Movement_related_firing_P_value_pfc.append(pfc_row['Movement_related_firing_P_value'].item())
        Reward_related_firing_P_value_pfc.append(pfc_row['Reward_related_firing_P_value'].item())
        BG_firing_CC_with_PPD_pfc.append(pfc_row['Background_firing_correlation_Coefficient_with_PPD'].item())
        BG_firing_CC_with_PPD_P_value_pfc.append(pfc_row['Background_firing_correlation_P_ value_with PPD'].item())
        Response_magnitude_CC_with_PPD_pfc.append(pfc_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
        Response_magnitude_CC_with_PPD_P_value_pfc.append(pfc_row['Response_magnitude_correlation_P_ value_with PPD'].item())

    
        spike_count_row = spike_count_data.loc[(spike_count_data['session']==session_name) & (spike_count_data['str']==str_name) & (spike_count_data['pfc']==pfc_name)]
        peak.append(spike_count_row['peak'].item())
        peak_width.append(spike_count_row['peak_width'].item())
        spike_count_in_peak.append(spike_count_row['counts_in_peak'].item())
    else:
        continue

result = pd.DataFrame(data={'mouse': mice, 'date': dates, 'str_name': str_names, 'pfc_name': pfc_names, 'Stimulus_related_firing_P_value_str': Stimulus_related_firing_P_value_str, 'Movement_related_firing_P_value_str': Movement_related_firing_P_value_str, 'Reward_related_firing_P_value_str': Reward_related_firing_P_value_str, 'BG_firing_CC_with_PPD_str': BG_firing_CC_with_PPD_str, 'BG_firing_CC_with_PPD_P_value_str': BG_firing_CC_with_PPD_P_value_str, 'Response_magnitude_CC_with_PPD_str': Response_magnitude_CC_with_PPD_str, 'Response_magnitude_CC_with_PPD_P_value_str': Response_magnitude_CC_with_PPD_P_value_str, 'Stimulus_related_firing_P_value_pfc': Stimulus_related_firing_P_value_pfc, 'Movement_related_firing_P_value_pfc': Movement_related_firing_P_value_pfc, 'Reward_related_firing_P_value_pfc': Reward_related_firing_P_value_pfc, 'BG_firing_CC_with_PPD_pfc': BG_firing_CC_with_PPD_pfc, 'BG_firing_CC_with_PPD_P_value_pfc': BG_firing_CC_with_PPD_P_value_pfc, 'Response_magnitude_CC_with_PPD_pfc': Response_magnitude_CC_with_PPD_pfc, 'Response_magnitude_CC_with_PPD_P_value_pfc': Response_magnitude_CC_with_PPD_P_value_pfc})

In [2]:
mice = []
dates = []
str_names = []
pfc_names = []

Stimulus_related_firing_P_value_str = []
Movement_related_firing_P_value_str = []
Reward_related_firing_P_value_str = []
BG_firing_CC_with_PPD_str = []
BG_firing_CC_with_PPD_P_value_str= []
Response_magnitude_CC_with_PPD_str = []
Response_magnitude_CC_with_PPD_P_value_str = []

Stimulus_related_firing_P_value_pfc = []
Movement_related_firing_P_value_pfc = []
Reward_related_firing_P_value_pfc = []
BG_firing_CC_with_PPD_pfc = []
BG_firing_CC_with_PPD_P_value_pfc = []
Response_magnitude_CC_with_PPD_pfc = []
Response_magnitude_CC_with_PPD_P_value_pfc = []

peak = []
peak_width = []
spike_count_in_peak = []

for qualified_image in listdir(pjoin('data', 'PMSE', 'qualified')):
    pair_name = qualified_image.split('.')[0]
    session_name, s, str_code, p, pfc_code = pair_name.split('_')
    spike_count_data = pd.read_csv(pjoin('data', 'PMSE', f'{session_name}.csv'))
    str_name = '_'.join([s, str_code])
    pfc_name = '_'.join([p, pfc_code])
    mouse = session_name[:6]
    date = session_name[6:]
    year = date[2:4]
    month = date[4:6]
    day = date[6:]
    session_name_wei = '_'.join([mouse, '.'.join([year, month, day])])

    str_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='STR') & (data['Cell ID']==int(str_code)+1)]
    pfc_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='PFC') & (data['Cell ID']==int(pfc_code)+1)]


    mice.append(mouse)
    dates.append(date)
    str_names.append(f'str_{int(str_code) + 1}')
    pfc_names.append(f'pfc_{int(pfc_code) + 1}')

    Stimulus_related_firing_P_value_str.append(str_row['Stimulus_related_firing_P_value'].item())
    Movement_related_firing_P_value_str.append(str_row['Movement_related_firing_P_value'].item())
    Reward_related_firing_P_value_str.append(str_row['Reward_related_firing_P_value'].item())
    BG_firing_CC_with_PPD_str.append(str_row['Background_firing_correlation_Coefficient_with_PPD'].item())
    BG_firing_CC_with_PPD_P_value_str.append(str_row['Background_firing_correlation_P_ value_with PPD'].item())
    Response_magnitude_CC_with_PPD_str.append(str_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
    Response_magnitude_CC_with_PPD_P_value_str.append(str_row['Response_magnitude_correlation_P_ value_with PPD'].item())

    Stimulus_related_firing_P_value_pfc.append(pfc_row['Stimulus_related_firing_P_value'].item())
    Movement_related_firing_P_value_pfc.append(pfc_row['Movement_related_firing_P_value'].item())
    Reward_related_firing_P_value_pfc.append(pfc_row['Reward_related_firing_P_value'].item())
    BG_firing_CC_with_PPD_pfc.append(pfc_row['Background_firing_correlation_Coefficient_with_PPD'].item())
    BG_firing_CC_with_PPD_P_value_pfc.append(pfc_row['Background_firing_correlation_P_ value_with PPD'].item())
    Response_magnitude_CC_with_PPD_pfc.append(pfc_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
    Response_magnitude_CC_with_PPD_P_value_pfc.append(pfc_row['Response_magnitude_correlation_P_ value_with PPD'].item())


    spike_count_row = spike_count_data.loc[(spike_count_data['session']==session_name) & (spike_count_data['str']==str_name) & (spike_count_data['pfc']==pfc_name)]
    peak.append(spike_count_row['peak'].item())
    peak_width.append(spike_count_row['peak_width'].item())
    spike_count_in_peak.append(spike_count_row['counts_in_peak'].item())


result = pd.DataFrame(data={'mouse': mice, 'date': dates, 'str_name': str_names, 'pfc_name': pfc_names, 'Stimulus_related_firing_P_value_str': Stimulus_related_firing_P_value_str, 'Movement_related_firing_P_value_str': Movement_related_firing_P_value_str, 'Reward_related_firing_P_value_str': Reward_related_firing_P_value_str, 'BG_firing_CC_with_PPD_str': BG_firing_CC_with_PPD_str, 'BG_firing_CC_with_PPD_P_value_str': BG_firing_CC_with_PPD_P_value_str, 'Response_magnitude_CC_with_PPD_str': Response_magnitude_CC_with_PPD_str, 'Response_magnitude_CC_with_PPD_P_value_str': Response_magnitude_CC_with_PPD_P_value_str, 'Stimulus_related_firing_P_value_pfc': Stimulus_related_firing_P_value_pfc, 'Movement_related_firing_P_value_pfc': Movement_related_firing_P_value_pfc, 'Reward_related_firing_P_value_pfc': Reward_related_firing_P_value_pfc, 'BG_firing_CC_with_PPD_pfc': BG_firing_CC_with_PPD_pfc, 'BG_firing_CC_with_PPD_P_value_pfc': BG_firing_CC_with_PPD_P_value_pfc, 'Response_magnitude_CC_with_PPD_pfc': Response_magnitude_CC_with_PPD_pfc, 'Response_magnitude_CC_with_PPD_P_value_pfc': Response_magnitude_CC_with_PPD_P_value_pfc})

result.to_csv('all_pairs.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/PMSE/qualified'

## Extract All Qualified Pairs

In [6]:
mice = []
dates = []
str_names = []
pfc_names = []

Stimulus_related_firing_P_value_str = []
Movement_related_firing_P_value_str = []
Reward_related_firing_P_value_str = []
BG_firing_CC_with_PPD_str = []
BG_firing_CC_with_PPD_P_value_str= []
Response_magnitude_CC_with_PPD_str = []
Response_magnitude_CC_with_PPD_P_value_str = []

Stimulus_related_firing_P_value_pfc = []
Movement_related_firing_P_value_pfc = []
Reward_related_firing_P_value_pfc = []
BG_firing_CC_with_PPD_pfc = []
BG_firing_CC_with_PPD_P_value_pfc = []
Response_magnitude_CC_with_PPD_pfc = []
Response_magnitude_CC_with_PPD_P_value_pfc = []

peak = []
peak_width = []
spike_count_in_peak = []

for qualified_image in listdir(pjoin('data', 'PMSE', 'qualified')):
    pair_name = qualified_image.split('.')[0]
    session_name, s, str_code, p, pfc_code = pair_name.split('_')
    spike_count_data = pd.read_csv(pjoin('data', 'PMSE', f'{session_name}.csv'))
    str_name = '_'.join([s, str_code])
    pfc_name = '_'.join([p, pfc_code])
    mouse = session_name[:6]
    date = session_name[6:]
    year = date[2:4]
    month = date[4:6]
    day = date[6:]
    session_name_wei = '_'.join([mouse, '.'.join([year, month, day])])

    str_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='STR') & (data['Cell ID']==int(str_code)+1)]
    pfc_row = data.loc[(data['session']==session_name_wei) & (data['Cell location']=='PFC') & (data['Cell ID']==int(pfc_code)+1)]

    if (str_row['Background_firing_correlation_P_ value_with PPD'].item() < 0.01 and 
       pfc_row['Background_firing_correlation_P_ value_with PPD'].item() < 0.01):
        mice.append(mouse)
        dates.append(date)
        str_names.append(f'str_{int(str_code) + 1}')
        pfc_names.append(f'pfc_{int(pfc_code) + 1}')

        Stimulus_related_firing_P_value_str.append(str_row['Stimulus_related_firing_P_value'].item())
        Movement_related_firing_P_value_str.append(str_row['Movement_related_firing_P_value'].item())
        Reward_related_firing_P_value_str.append(str_row['Reward_related_firing_P_value'].item())
        BG_firing_CC_with_PPD_str.append(str_row['Background_firing_correlation_Coefficient_with_PPD'].item())
        BG_firing_CC_with_PPD_P_value_str.append(str_row['Background_firing_correlation_P_ value_with PPD'].item())
        Response_magnitude_CC_with_PPD_str.append(str_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
        Response_magnitude_CC_with_PPD_P_value_str.append(str_row['Response_magnitude_correlation_P_ value_with PPD'].item())

        Stimulus_related_firing_P_value_pfc.append(pfc_row['Stimulus_related_firing_P_value'].item())
        Movement_related_firing_P_value_pfc.append(pfc_row['Movement_related_firing_P_value'].item())
        Reward_related_firing_P_value_pfc.append(pfc_row['Reward_related_firing_P_value'].item())
        BG_firing_CC_with_PPD_pfc.append(pfc_row['Background_firing_correlation_Coefficient_with_PPD'].item())
        BG_firing_CC_with_PPD_P_value_pfc.append(pfc_row['Background_firing_correlation_P_ value_with PPD'].item())
        Response_magnitude_CC_with_PPD_pfc.append(pfc_row['Response_magnitude_correlation_Coefficient_with_PPD'].item())
        Response_magnitude_CC_with_PPD_P_value_pfc.append(pfc_row['Response_magnitude_correlation_P_ value_with PPD'].item())

    
        spike_count_row = spike_count_data.loc[(spike_count_data['session']==session_name) & (spike_count_data['str']==str_name) & (spike_count_data['pfc']==pfc_name)]
        peak.append(spike_count_row['peak'].item())
        peak_width.append(spike_count_row['peak_width'].item())
        spike_count_in_peak.append(spike_count_row['counts_in_peak'].item())
    else:
        continue

result = pd.DataFrame(data={'mouse': mice, 'date': dates, 'str_name': str_names, 'pfc_name': pfc_names, 'Stimulus_related_firing_P_value_str': Stimulus_related_firing_P_value_str, 'Movement_related_firing_P_value_str': Movement_related_firing_P_value_str, 'Reward_related_firing_P_value_str': Reward_related_firing_P_value_str, 'BG_firing_CC_with_PPD_str': BG_firing_CC_with_PPD_str, 'BG_firing_CC_with_PPD_P_value_str': BG_firing_CC_with_PPD_P_value_str, 'Response_magnitude_CC_with_PPD_str': Response_magnitude_CC_with_PPD_str, 'Response_magnitude_CC_with_PPD_P_value_str': Response_magnitude_CC_with_PPD_P_value_str, 'Stimulus_related_firing_P_value_pfc': Stimulus_related_firing_P_value_pfc, 'Movement_related_firing_P_value_pfc': Movement_related_firing_P_value_pfc, 'Reward_related_firing_P_value_pfc': Reward_related_firing_P_value_pfc, 'BG_firing_CC_with_PPD_pfc': BG_firing_CC_with_PPD_pfc, 'BG_firing_CC_with_PPD_P_value_pfc': BG_firing_CC_with_PPD_P_value_pfc, 'Response_magnitude_CC_with_PPD_pfc': Response_magnitude_CC_with_PPD_pfc, 'Response_magnitude_CC_with_PPD_P_value_pfc': Response_magnitude_CC_with_PPD_P_value_pfc})

result.to_csv('strong_correlation_pairs_ITI.csv')